In [1]:
import timeit
import numpy as np

In [26]:
result_sets = np.logspace(0, 6, 7)

In [28]:
from drillpy import PyQuery
import timeit

def run(connString):
    def doRun():
        R1 = PyQuery()
        df = R1.get_cols(connString)
        return df
    return doRun

native_times = {}
for results in result_sets:
    connString = {'query' : "select * from dfs.flights.flights_by_yr limit {}".format(results), 
                  'type': 'sql', 
                  'connectStr' : 'local=172.17.0.2:31010',
                  'api' : 'async',
                  'logLevel' : 'error'}
    result = run(connString)
    native_times[results] = timeit.Timer(run(connString)).repeat(repeat=3, number=1)

In [29]:
native_times

{1.0: [0.9279497339994123, 0.685328623996611, 0.6460083249985473],
 10.0: [0.7054430649986898, 1.3593373850017088, 0.9528090790008719],
 100.0: [0.9339452410022204, 0.8913531949983735, 0.6835905640000419],
 1000.0: [0.8156201509991661, 0.750704031001078, 0.8666444820009929],
 10000.0: [0.6871445959986886, 0.7085528930001601, 1.5302053099985642],
 100000.0: [1.3375563039990084, 1.2616593450002256, 1.189920541997708],
 1000000.0: [7.392612472001929, 5.769677295000292, 5.555417019000743]}

In [30]:
from turbodbc import connect, make_options, Megabytes
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
options = make_options(read_buffer_size=Megabytes(20),
                                           autocommit=True)

conn = connect(dsn='drill', turbodbc_options=options)

turbo_times = {}
for results in result_sets:
    sql = 'select * from dfs.flights.flights_by_yr limit {}'.format(results)
    fun = lambda: conn.cursor().execute(sql).fetchallarrow().to_pandas()
    turbo_times[results] = timeit.Timer(fun).repeat(repeat=3, number=1)

In [ ]:
turbo_times

{1.0: [1.551988088998769, 1.5291844490020594, 1.541293129997939],
 10.0: [1.688088731996686, 1.4894256410007074, 1.602607671997248],
 100.0: [1.3738734559992736, 1.2308062919983058, 1.3236386429998674],
 1000.0: [1.4293271410024317, 1.5005177560015, 1.437558279998484],
 10000.0: [1.654775406997942, 1.49588034000044, 1.6554415629980213],
 100000.0: [2.1380584449980233, 2.1837525480004842, 2.3455752480003866],
 1000000.0: [8.006138142001873, 7.352187275999313, 9.92592518499805]}

In [ ]:
turbo_times_dict = {}
for results in result_sets:
    sql = 'select * from dfs.flights.flights_by_yr limit {}'.format(results)
    fun = lambda: conn.cursor().execute(sql).fetchallarrow(strings_as_dictionary=True).to_pandas()
    turbo_times_dict[results] = timeit.Timer(fun).repeat(repeat=3, number=1)

In [ ]:
turbo_times_dict